___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

## Set Leverage for an equity

We can simply use **security.SetLeverage(leverage)** in order to allow leverage for a given equity

In [ ]:
class Leverage(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)  # We want to invest on the first January 2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.apple.SetLeverage(2)  # Use a 2x leverage
        
    def OnData(self, data):
        
        if not self.Portfolio.Invested:
            self.SetHoldings(self.apple.Symbol, 2)


In [ ]:
class Leverage(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)  # We want to invest on the first January 2010
        self.SetCash(10000)  #  10,000$ are available
        self.SetBrokerageModel(BrokerageName.InteractiveBrokersBrokerage, AccountType.Cash)
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
         
        
    def OnData(self, data):
        
        if not self.Portfolio.Invested:
            self.SetHoldings(self.apple.Symbol, 1)


## Shorting
We will take a look at two examples:
First we will short Seaworld in January 2020 and then close the position in May.
Second we will again short Seaworld in January 2020 but dont close the position.
What would be the better action?

In [ ]:
class Shorting(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)  # We want to invest on the first January 2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.seaworld = self.AddEquity("SEAS", Resolution.Daily)
        self.invest = True
        
    def OnData(self, data):
        
        # Short as may stocks as possible given the current price and available cash
        if  not self.Portfolio.Invested and self.invest:
            self.MarketOrder(self.seaworld.Symbol, -self.Portfolio.Cash / self.seaworld.Close )
            
        # Liquidate position in May and dont invest anymore
        if self.Time.month == 5:
            self.Liquidate(self.seaworld.Symbol)
            self.invest = False


In [ ]:
class ShortingNeverLiquidate(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)  # We want to invest on the first January 2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.seaworld = self.AddEquity("SEAS", Resolution.Daily)
        self.invest = True
        
    def OnData(self, data):
        
        # Short as may stocks as possible given the current price and available cash
        if  not self.Portfolio.Invested and self.invest:
            self.MarketOrder(self.seaworld.Symbol, -self.Portfolio.Cash / self.seaworld.Close )

## Margin Call
 

In [ ]:
class MarginCallEventsAlgorithm(QCAlgorithm):
    """
    This algorithm showcases two margin related event handlers.
    OnMarginCallWarning: Fired when a portfolio's remaining margin dips below 5% of the total portfolio value
    OnMarginCall: Fired immediately before margin call orders are execued, this gives the algorithm a change to regain margin on its own through liquidation
    """

    def Initialize(self):

        self.SetCash(100000)
        self.SetStartDate(2020,1,1)
        self.SetEndDate(2020,5,1)
        self.AddEquity("SPY", Resolution.Daily)
        # cranking up the leverage increases the odds of a margin call
        # when the security falls in value
        self.Securities["SPY"].SetLeverage(10)

    def OnData(self, data):
        if not self.Portfolio.Invested:
            self.SetHoldings("SPY",10)

    def OnMarginCall(self, requests):

        # Margin call event handler. This method is called right before the margin call orders are placed in the market.
        # <param name="requests">The orders to be executed to bring this algorithm within margin limits</param>
        # this code gets called BEFORE the orders are placed, so we can try to liquidate some of our positions
        # before we get the margin call orders executed. We could also modify these orders by changing their quantities
        for order in requests:

            # liquidate an extra 10% each time we get a margin call to give us more padding
            newQuantity = int( order.Quantity * 1.1)
            requests.remove(order)
            requests.append(SubmitOrderRequest(order.OrderType, order.SecurityType, order.Symbol, newQuantity, order.StopPrice, order.LimitPrice, self.Time, "OnMarginCall"))

        return requests

    def OnMarginCallWarning(self):

        # Margin call warning event handler.
        # This method is called when Portfolio.MarginRemaining is under 5% of your Portfolio.TotalPortfolioValue
        # a chance to prevent a margin call from occurring

        spyHoldings = self.Securities["SPY"].Holdings.Quantity
        shares = int(-spyHoldings * 0.005)
        self.Error("{0} - OnMarginCallWarning(): Liquidating {1} shares of SPY to avoid margin call.".format(self.Time, shares))
        self.MarketOrder("SPY", shares)